# Lesson2 畳み込みニューラルネットワーク (CNN)

## 目次

- Section1 解説
  - 1.1 CNN基礎
  - 1.2 Convolution(畳み込み)層
  - 1.3 Pooling(プーリング)層
  - 1.4 確認問題

- Section2 実装①
  - 2.1 Fasion MNISTをCNNでクラス分類
  - 2.2 CIFAR10のデータをCNNでクラス分類
  
- Section3 テクニック・発展内容
  - 3.1 Data Augmentation
  - 3.2 画像データの正規化
  - 3.3 Batch Normalization
  - 3.4 Skip Connection  (Residual Network)
  - 3.5 学習済みネットワークの利用
  - 3.6 学習させたモデルの保存・再利用
  - 3.7 確認問題

- Section4 実装②
  - 4.1 CIFAR10のデータをCNNでクラス分類②

- Section5 ケーススタディ

- Homework

## 3.7 の解答

問1: ② 問2: ① 問3: ① 問4: ①

In [1]:
%matplotlib inline

import os

import keras
import numpy as np
import matplotlib.pyplot as plt

from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input, Activation, add, Add, Dropout
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.datasets import cifar10
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

random_state = 42

Using TensorFlow backend.


## Section4 実装②

### 4.1 CIFAR10のデータをCNNでクラス分類②

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32') / 255
y_train = np.eye(10)[y_train.astype('int32').flatten()]

x_test = x_test.astype('float32') / 255
y_test = np.eye(10)[y_test.astype('int32').flatten()]

x_train, x_valid, y_train, y_valid = train_test_split(
    x_train, y_train, test_size=10000)

Section3の学習内容も踏まえて、CIFAR10のクラス分類を行いたいと思います。

まず、モデルの作成を行います。

In [3]:
model = Sequential()

model.add(Conv2D(6, kernel_size=(5, 5), activation='relu',
                 kernel_initializer='he_normal', input_shape=(32, 32, 3)))  # 32x32x3 -> 28x28x6
model.add(MaxPooling2D(pool_size=(2, 2)))  # 28x28x6 -> 14x14x6
model.add(Conv2D(16, kernel_size=(5, 5), activation='relu',
                 kernel_initializer='he_normal'))  # 14x14x6 -> 10x10x16
model.add(MaxPooling2D(pool_size=(2, 2)))  # 10x10x16 -> 5x5x16

model.add(Flatten())  # 5x5x16 -> 400
model.add(Dense(120, activation='relu',
                kernel_initializer='he_normal'))  # 400 ->120
model.add(Dense(84, activation='relu', kernel_initializer='he_normal'))  # 120 ->84
model.add(Dense(10, activation='softmax'))  # 84 ->10

model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy']
)

次に、Section3で学習したDataAugumentationや画像データの正規化を学習に反映させてみます。

kerasのImageDataGeneratorを使用します。

In [4]:
datagen = ImageDataGenerator(
    width_shift_range=0.2,  # 3.1.1 左右にずらす
    height_shift_range=0.2,  # 3.1.2 上下にずらす
    horizontal_flip=True,  # 3.1.3 左右反転
    # 3.2.1 Global Contrast Normalization (GCN) (Falseに設定しているのでここでは使用していない)
    samplewise_center=False,
    samplewise_std_normalization=False,
    zca_whitening=False)  # 3.2.2 Zero-phase Component Analysis (ZCA) Whitening (Falseに設定しているのでここでは使用していない)

In [5]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=100),
                    steps_per_epoch=x_train.shape[0] // 100, epochs=30, validation_data=(x_valid, y_valid))

Epoch 1/30
400/400 [==============================] - 19s 48ms/step - loss: 1.8601 - acc: 0.3128 - val_loss: 1.6124 - val_acc: 0.4134
Epoch 2/30
400/400 [==============================] - 15s 37ms/step - loss: 1.6443 - acc: 0.3940 - val_loss: 1.4765 - val_acc: 0.4705
Epoch 3/30
400/400 [==============================] - 15s 37ms/step - loss: 1.5759 - acc: 0.4279 - val_loss: 1.4583 - val_acc: 0.4709
Epoch 4/30
400/400 [==============================] - 15s 37ms/step - loss: 1.5140 - acc: 0.4543 - val_loss: 1.3853 - val_acc: 0.5003
Epoch 5/30
400/400 [==============================] - 15s 38ms/step - loss: 1.4658 - acc: 0.4707 - val_loss: 1.3801 - val_acc: 0.4963
Epoch 6/30
400/400 [==============================] - 15s 37ms/step - loss: 1.4386 - acc: 0.4824 - val_loss: 1.3246 - val_acc: 0.5284
Epoch 7/30
400/400 [==============================] - 15s 37ms/step - loss: 1.4027 - acc: 0.4949 - val_loss: 1.2996 - val_acc: 0.5389
Epoch 8/30
400/400 [==============================] - 15s 37ms

In [8]:
x_train.shape[0] // 100

400